In [1]:
import numpy as np
import pandas as pd
import os
import warnings

from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
warnings.filterwarnings('ignore')
%matplotlib inline

## Data

In [2]:
url="http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/AMAZON_FASHION.csv"
dataset=pd.read_csv(url, header=None)
dataset.columns = ['item','user','rating','timestamp']
dataset['user_id'] = dataset[['user']].sum(axis=1).map(hash)
dataset['item_id'] = dataset[['item']].sum(axis=1).map(hash)
dataset

,item,user,rating,timestamp,user_id,item_id
0,7106116521,A1D4G1SNUZWQOT,5.0,1413763200,-1497349159846488537,-6381021208221860324
1,7106116521,A3DDWDH9PX2YX2,2.0,1411862400,1180494009815071824,-6381021208221860324
2,7106116521,A2MWC41EW7XL15,4.0,1408924800,-6080867210258812294,-6381021208221860324
3,7106116521,A2UH2QQ275NV45,2.0,1408838400,2879706517304461579,-6381021208221860324
4,7106116521,A89F3LQADZBS5,3.0,1406419200,-1259316630074921148,-6381021208221860324
...,...,...,...,...,...,...
883631,B01HJHTH5U,A1ZSB2Q144UTEY,5.0,1487635200,-2848563161803962499,5426872683538826224
883632,B01HJHTH5U,A2CCDV0J5VB6F2,5.0,1480032000,-2095503269209032846,5426872683538826224
883633,B01HJHTH5U,A3O90PACS7B61K,3.0,1478736000,2948444125602717888,5426872683538826224
883634,B01HJHF97K,A2HO94I89U3LNH,3.0,1478736000,2899278387749796472,2565274136055259015


## Define NN

In [3]:
train, test = train_test_split(dataset, test_size=0.2)

In [6]:
# creating item embedding path
item_input = Input(shape=[1], name="Items-Input")
item_embedding = Embedding(len(train.item)+1, 10, name="Items-Embedding")(item_input)
item_vec = Flatten(name="Flatten-Items")(item_embedding)
# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(len(train.user)+1, 10, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
# concatenate features
conc = Concatenate()([item_vec, user_vec])
# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(32, activation='relu')(fc2)
out = Dense(1)(fc3)
# Create model and compile it
model = Model([item_input, user_input], out)
model.compile('adam', 'mean_squared_error')

## Train NN

In [7]:
history = model.fit([train.item_id, train.user_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
22091/22091 [==============================] - 381s 17ms/step - loss: 3.0605
Epoch 2/5
22091/22091 [==============================] - 382s 17ms/step - loss: 2.0106
Epoch 3/5
22091/22091 [==============================] - 381s 17ms/step - loss: 2.0105
Epoch 4/5
22091/22091 [==============================] - 383s 17ms/step - loss: 2.0106
Epoch 5/5
22091/22091 [==============================] - 383s 17ms/step - loss: 2.0106


## Predictions NN

In [17]:
k = 15
predictions = model.predict([test.item_id.head(k), test.user_id.head(k)])
[print(predictions[i], test.rating.iloc[i]) for i in range(k)]

[3.9052536] 5.0
[3.9052536] 2.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 4.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 4.0
[3.9052536] 5.0
[3.9052536] 5.0
[3.9052536] 1.0
[3.9052536] 1.0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]